In [260]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model, model_selection
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

import seaborn as sns
from sklearn.metrics.pairwise import pairwise_kernels

import scipy

In [261]:
B = pd.read_parquet("./data/B_HKML.parquet")

In [251]:
all_epochs = B.date.unique()

In [252]:
epochs = all_epochs[:1]

In [253]:
epochs

array(['2016-02-17T00:00:00.000000000'], dtype='datetime64[ns]')

In [254]:
i = 0
j = 0
for epoch in epochs:
    B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T
    i += B_np.shape[0]
    j += B_np.shape[1]

In [255]:
Big_B = np.zeros((i, j))

In [256]:
i = 0
j = 0
for epoch in epochs:
    B_np = B[B["date"] == epoch].drop("date", axis=1).to_numpy().T

    di = B_np.shape[0]
    dj = B_np.shape[1]

    Big_B[i : i + di, j : j + dj] = B_np

    i += di
    j += dj

In [257]:
B = Big_B

In [258]:
# plt.figure(figsize=(15, 10))
# sns.heatmap(np.abs(B))
# plt.show()

In [259]:
B.shape

(85, 996)

In [232]:
X_df = pd.read_parquet("./data/X_HKML.parquet")
X = X_df[X_df["date"].isin(epochs)].drop("date", axis=1).to_numpy()
X.shape

(996, 160)

In [233]:
y_df = pd.read_parquet("./data/Y_HKML.parquet")
y = y_df.iloc[: len(X)].to_numpy().ravel()
y.shape

(996,)

In [234]:
C = 0.0
D = 100.0

In [247]:
K = pairwise_kernels(X, metric="rbf")
# plt.figure(figsize=(15, 10))
# sns.heatmap(K)
# plt.show()

In [248]:
BBK = B.T @ B @ K

In [246]:
# plt.figure(figsize=(15, 10))
# sns.heatmap(BBK)
# plt.show()

In [238]:
M = K + C * np.identity(K.shape[0]) + D * BBK

In [239]:
M.shape

(996, 996)

In [240]:
# plt.figure(figsize=(15, 10))
# sns.heatmap(M)
# plt.show()

In [241]:
dual_coef = np.linalg.solve(M, y)

In [242]:
dual_coef.shape

(996,)

# Predict

In [189]:
Xp = X

In [190]:
Kp = pairwise_kernels(Xp, X, metric="rbf")

In [191]:
yp = Kp @ dual_coef

In [192]:
exp_vec = B @ yp
exp = np.dot(exp_vec, exp_vec)
exp

4.6013714345243515e-05

In [193]:
mse = np.dot(y - yp, y - yp)
mse

0.7410185754913977

In [194]:
ridge = dual_coef @ Kp @ dual_coef
ridge

18.38540857739536

In [195]:
y_hat_df = pd.DataFrame()
y_hat_df["date"] = X_df[X_df["date"].isin(epochs)].date
y_hat_df["y_hat"] = yp
y_hat_df["Y"] = y

In [196]:
y_hat_df.groupby("date").corr()

y_hat         Y
date                                
2016-02-17 y_hat  1.000000  0.961465
           Y      0.961465  1.000000
2016-02-24 y_hat  1.000000  0.959146
           Y      0.959146  1.000000
2016-03-02 y_hat  1.000000  0.973758
           Y      0.973758  1.000000
2016-03-09 y_hat  1.000000  0.955864
           Y      0.955864  1.000000
2016-03-16 y_hat  1.000000  0.959274
           Y      0.959274  1.000000
2016-03-23 y_hat  1.000000  0.961027
           Y      0.961027  1.000000
2016-03-30 y_hat  1.000000  0.937559
           Y      0.937559  1.000000
2016-04-06 y_hat  1.000000  0.963502
           Y      0.963502  1.000000